In [1]:
from PIL import Image
import h5py
import anubisPlotUtils as anPlot
import json
import numpy as np
import os
import hist as hi
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc.
import mplhep as hep
hep.style.use([hep.style.ATLAS])
import sys
import AnalysisToolAnubis as AT
import pandas as pd
import tkinter as tk
from tkinter import ttk
from pandasgui import show
from matplotlib.backends.backend_pdf import PdfPages
from scipy.signal import convolve2d
from scipy.ndimage import label, find_objects
from matplotlib.backends.backend_pdf import PdfPages
import ipywidgets as widgets
from IPython.display import display


ERROR:root:Cannot activate multiple GUI eventloops


In [2]:
#Importing the data
thisData = AT.importDatafile('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubisData\\60sRun_24_3_4.h5')


In [3]:
mapping = {
    0: {
        (0, 31): ('rpc3', 'eta'),
        (32, 63): ('rpc3', 'phi1'),
        (64, 95): ('rpc3', 'phi2'),
        (96, 127): ('rpc2', 'eta'),
    },
    1: {
        (0, 31): ('rpc2', 'phi1'),
        (32, 63): ('rpc2', 'phi2'),
        (64, 95): ('rpc1', 'eta'),
        (96, 127): ('rpc1', 'phi1'),
    },
    2: {
        (0, 31): ('rpc1', 'phi2'),
        (32, 63): ('rpc4', 'eta'),
        (64, 95): ('rpc4', 'phi1'),
        (96, 127): ('rpc4', 'phi2'),
    },
    3: {
        (0, 31): ('rpc6', 'eta'),
        (32, 63): ('rpc6', 'phi1'),
        (64, 95): ('rpc6', 'phi2'),
        (96, 127): ('rpc5', 'eta'),
    },
    4: {
        (0, 31): ('rpc5', 'phi1'),
        (32, 63): ('rpc5', 'phi2'),
    },
}

In [4]:
def assign_to_coordinates(tdc, channel, mapping):
    for channel_range, (rpc, side) in mapping.get(tdc, {}).items():
        if channel_range[0] <= channel <= channel_range[1]:
            offset = channel - channel_range[0]
            if side == 'eta':
                return [rpc, 'eta', [0, offset]]
            elif side in ['phi1', 'phi2']:
                return [rpc, 'phi', [offset, 0]]
    return ['Null', 'Null', [tdc, channel]]  # If no match found

In [5]:
def remake_data(Data, mapping):
    full_data = []
    for tdc in range(0, len(Data)):
        for events in range(0, len(Data[tdc])): 
            for hits in Data[tdc][events]:
                channel = (hits>>24)&0x7f
                coord = assign_to_coordinates(tdc, channel, mapping)
                event_time = hits&0xfffff
                
            
                
                full_data.append({
                    'tdc': tdc,
                    'event number': events,
                    'channel':channel,
                    'rpc number': coord[0],
                    'strip direction': coord[1],
                    'hit coordinate': coord[2],
                    'time': event_time
                })
    df = pd.DataFrame(full_data)
        
    return df
    

In [6]:
df = remake_data(thisData, mapping)


In [22]:
show(df)


PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


The instruction was unclear, what does it meant by three coincidence required in the email? possibly three different RPC hit required for the trigger to record... The reason of the event time being mostly around 200 arbitrary value, is probably the time takes for the computer to react and record the correcponding time, I won't trust that value to be the exact time, but it will be interesting to see if there are correlation between those time and rpc number

In [7]:
event_rpc_dict = {}

for event in df['event number'].unique():
    rpcs_involved = df.loc[df['event number'] == event, 'rpc number'].unique()
    event_rpc_dict[event] = rpcs_involved

events_with_less_than_3_rpcs = 0

for event, rpcs in event_rpc_dict.items():
    if len(rpcs) < 3:
        events_with_less_than_3_rpcs += 1

print(events_with_less_than_3_rpcs)

8


In [8]:
event_counts = df['event number'].value_counts()

events_less_than_3_occurrences = event_counts[event_counts < 3]

num_events_less_than_3 = len(events_less_than_3_occurrences)

print(num_events_less_than_3)

0


Hence it might not have been between rpcs, try if it is between number of simulateous events
Well, that is self explainatary, and since we have 19473 entries, each event contains roughly 30.19 entries, didn't expect to be this tbh

Well it doesn't stop data analysis
total 576 strips, 19473 interaction recorded, if all through, this is about 0.56 Hz of dark counts, which is slightly lower than the expected 1Hz.

Checking if there are any broken channels (or strips)

In [9]:
all_combinations = []
for tdc in range(5):  # tdc0 to tdc4
    tdcnumber = tdc
    max_channel = 63 if tdc == 4 else 127
    for channel in range(max_channel + 1):
        all_combinations.append((tdc, channel))

all_combinations_df = pd.DataFrame(all_combinations, columns=['tdc', 'channel'])
missing_combinations = pd.merge(all_combinations_df, df, on=['tdc', 'channel'], how='left', indicator=True)
missing_combinations = missing_combinations[missing_combinations['_merge'] == 'left_only']
missing_combinations = missing_combinations[['tdc', 'channel']]
show(missing_combinations)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI


INFO:pandasgui.gui:Opening PandasGUI


tdc0 channel 31
tdc0 channel 107
tdc0 channel 111
tdc2 channel 41
tdc3 channel 51
These never fired, for a 30 second run and if the dark count is stochastic with poisson distribution, it is very unlikely this will happen, hence they are probably dead...

In [10]:
#slightly different way to store data, but does the same thing, it goes like rpc - event - strip
etaHits, phiHits = AT.divideHitCountsByRPC(thisData)

In [11]:
def countCoincidencesEvents(primRpc,SecRpc, winSize=5, minHit=3, maxHit=10):
    coincArray = [0 for channel in range(len(primRpc[0]))]
    for idx, event in enumerate(primRpc):
        for channel in range(len(event)):
            nHits = 0
            for itr in range(channel-int(winSize/2.), channel+int(winSize/2.)+1):
                if itr>=0 and itr<len(event):
                    if event[channel]>0:
                        nHits = nHits+1
            if nHits>=minHit and sum(event)<=maxHit:
                if sum(SecRpc[idx])>=minHit and sum(SecRpc[idx])<=maxHit:
                    coincArray[channel]=coincArray[channel]+1
    return coincArray

In [12]:
coincidence = {}
#The countCoincidences defined a window sieze, in which it says if the number of events within this window size is more than 3 and less than 10 hits occured per event (a cluster), 
# and if the second strip also had a a total event more than 3 and less than 10, 
# then it is recorded as a successful coincidence
for i in range(5):
    result_key = f'etaHits{i}_phiHits{i}' 
    coincidence[result_key] = AT.countCoincidences(etaHits[i], phiHits[i])
    coincidence[result_key] = AT.countCoincidences(phiHits[i], etaHits[i])
#This is useful as it gives an idea how many successful hits were observed at whatever places
print(coincidence)

{'etaHits0_phiHits0': [23, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 4, 5, 8, 6, 2, 7, 7, 4, 8, 11, 7, 4, 4, 6, 8, 6, 3, 4, 6, 7, 9, 15, 15, 11, 9, 6, 6, 2, 2, 0, 6, 2, 7, 6, 6, 7, 4, 3, 7, 4, 4, 5, 5, 3, 5, 5, 6, 6], 'etaHits1_phiHits1': [4, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 2, 5, 4, 1, 6, 6, 4, 5, 5, 6, 9, 7, 7, 9, 5, 5, 2, 2, 1, 0, 2, 1, 2, 3, 3, 7, 7, 2, 3, 3, 3, 2, 1, 4, 7, 3, 11, 10, 9, 5, 4, 6, 10, 4, 9, 7, 8, 8, 7, 12, 11, 7], 'etaHits2_phiHits2': [24, 5, 5, 4, 4, 2, 5, 4, 6, 7, 8, 9, 6, 6, 5, 7, 10, 9, 6, 6, 3, 5, 5, 5, 5, 5, 6, 4, 3, 3, 6, 4, 8, 5, 7, 6, 7, 9, 5, 13, 8, 9, 10, 8, 2, 2, 6, 3, 12, 8, 9, 7, 4, 3, 6, 5, 11, 6, 9, 6, 6, 6, 8, 5], 'etaHits3_phiHits3': [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 3, 2, 1, 0, 1, 1, 2, 2, 2, 2, 0, 0, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0], 'etaHits4_phiHits4': [9, 3, 4, 3, 4, 3, 2, 2, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 3, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 3,

Looking first at 1D data for the strippes, without any time associated with it (treating events as independant discreet series)
we can quantify a clustering metric

In [13]:
def calculate_cluster_metrics(event, minHit=3, maxHit=10):
    cluster_sizes = []
    cluster_locations = []
    current_cluster_size = 0
    start_index = None  
    
    for index, i in enumerate(event):
        if i != 0:
            current_cluster_size += i
            if start_index is None:
                start_index = index  
        elif i == 0 and current_cluster_size >= minHit and current_cluster_size < maxHit:
            cluster_midpoint = (start_index + index + 1) / 2.0  
            cluster_locations.append(cluster_midpoint)
            cluster_sizes.append(current_cluster_size)
            current_cluster_size = 0
            start_index = None 
        elif i == 0:
            current_cluster_size = 0
            start_index = None 
            
    # Handle the last cluster if it meets the criteria
    if current_cluster_size >= minHit and current_cluster_size < maxHit:
        cluster_midpoint = (start_index + len(event) + 1) / 2.0 
        cluster_locations.append(cluster_midpoint)
        cluster_sizes.append(current_cluster_size)
    
    return cluster_locations, cluster_sizes


In [14]:
columns = ['RPC Number', 'Event Number', 'cluster_locations', 'Cluster_size']
dfcluseta = pd.DataFrame(columns=columns)
dfclusphi = pd.DataFrame(columns=columns)

for i, rpc_data in enumerate(etaHits):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        for _ in range(len(cluster_sizes)):
            
            dfcluseta = dfcluseta.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

for i, rpc_data in enumerate(phiHits):
    for j, event in enumerate(rpc_data):
        cluster_locations, cluster_sizes = calculate_cluster_metrics(event)
        for _ in range(len(cluster_sizes)):
            
            dfclusphi = dfclusphi.append({'RPC Number': int(i), 'Event Number': int(j), 'cluster_locations': cluster_locations[_], 'Cluster_size': int(cluster_sizes[_])}, ignore_index=True)

C:\Users\Peter\AppData\Local\Temp\ipykernel_8792\1678047686.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_8792\1678047686.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_8792\1678047686.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_8792\1678047686.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Peter\AppData\Local\Temp\ipykernel_8792\1678047686.py:10: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat 

In [126]:
show(dfclusphi)

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [15]:
rpc_dfseta = {rpc: dfcluseta[dfcluseta['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfcluseta['RPC Number'].unique()}
rpc_dfsphi = {rpc: dfclusphi[dfclusphi['RPC Number'] == rpc].reset_index(drop=True) for rpc in dfclusphi['RPC Number'].unique()}

def plot_rpc_histograms(rpc_dfseta, rpc_dfsphi):
    with PdfPages('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\rpc_cluster_analysis.pdf') as pdf:
        for rpc in rpc_dfseta.keys():
            df_eta = rpc_dfseta[rpc]
            df_phi = rpc_dfsphi[rpc]
            for df, dataset_name in zip([df_eta, df_phi], ['eta', 'phi']):
                all_cluster_locations = []
                all_cluster_sizes = []
                
                for _, row in df.iterrows():
                    locs = row['cluster_locations']
                    sizes = row['Cluster_size']
                    
                    if not isinstance(locs, list):
                        locs = [locs] if not pd.isnull(locs) else []
                    if not isinstance(sizes, list):
                        sizes = [sizes] if not pd.isnull(sizes) else []
                    
                    all_cluster_locations.extend(locs)
                    all_cluster_sizes.extend(sizes)
                
                locations_series = pd.Series(all_cluster_locations)
                sizes_series = pd.Series(all_cluster_sizes)
                
                location_frequencies = locations_series.value_counts().sort_index()
                size_frequencies = sizes_series.value_counts().sort_index()
                
                fig, axs = plt.subplots(1, 2, figsize=(12, 6))
                fig.suptitle(f'RPC {rpc} - {dataset_name.upper()}')

                if not location_frequencies.empty:
                    axs[0].bar(location_frequencies.index, location_frequencies.values, color='skyblue', edgecolor='black')
                axs[0].set_title(f'Cluster Locations')
                axs[0].set_xlabel('Cluster Location')
                axs[0].set_ylabel('Frequency')

                if not size_frequencies.empty:
                    axs[1].bar(size_frequencies.index, size_frequencies.values, color='salmon', edgecolor='black')
                axs[1].set_title(f'Cluster Sizes')
                axs[1].set_xlabel('Cluster Size')
                axs[1].set_ylabel('Frequency')
                
                if dataset_name == 'phi':
                    axs[0].set_xlim(0, 64)
                    # axs[1].set_xlim(0, 64)
                if dataset_name == 'eta':
                    axs[0].set_xlim(0, 32)
                    # axs[1].set_xlim(0, 32)
                    
                plt.tight_layout(rect=[0, 0.03, 1, 0.95])
                pdf.savefig(fig)
                plt.close()


In [17]:
confirmation = input("This action might cause a large amount of folders to be created at unwanted areas in your computer, please make sure you have changed the pdf directory before proceeding. Are you ready to proceed? (yes/no):")


if confirmation.lower() == 'yes':
    print("Proceeding with the operation...")
    plot_rpc_histograms(rpc_dfseta, rpc_dfsphi)

else:
    print("Operation canceled.")

Proceeding with the operation...


Cool, now we can extend this into 2D, and try to find circles around interesting points, again, concerned about locations and size. Maybe we should define a kernel?
Being a good human being, I will again consider one event without thinking event time
and assume everything 100 efficient, if 2 strips cross, that is the coordinate of the hit

In [16]:
def create_hit_matrices(phiHits, etaHits):
    num_rpcs, num_events = 6, len(phiHits[0])
    
    hit_matrices = [[None for _ in range(num_events)] for _ in range(num_rpcs)]
    
    for z in range(num_rpcs):
        for l in range(num_events):
            phi_activations = phiHits[z][l]
            eta_activations = etaHits[z][l]
            
            hit_matrix = np.zeros((64, 32), dtype=int)
            for y in range(64):
                for x in range(32):
                    if phi_activations[y] == 1 and eta_activations[x] == 1:
                        hit_matrix[y, x] = 1
                        
            # Store the constructed hit matrix
            hit_matrices[z][l] = hit_matrix
            
    return hit_matrices

hit_matrix = create_hit_matrices(phiHits, etaHits)
print(hit_matrix[0][0])

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [17]:
def refine_clusters(hit_matrix, kernel_size, leniency):
    """
    Refine cluster identification to calculate the midpoint and rough size of each cluster.
    
    Parameters:
    - hit_matrix: 2D numpy array representing hits in the RPC detector.
    - kernel_size: Tuple indicating the dimensions of the kernel (height, width).
    - leniency: Percentage of the kernel that needs to be filled to consider it a cluster (0-1).
    
    Returns:
    - A list of tuples where each tuple represents a cluster's (midpoint row, midpoint column, cluster size).
    
    cluster size definition is not very clear, in this code, it is the number of activated points (hits) within the cluster
    which is less useful than the area but I don't know how to do area, there must be a package
    """
    
    kernel = np.ones(kernel_size)
    conv_result = convolve2d(hit_matrix, kernel, mode='same', boundary='fill', fillvalue=0)
    
    threshold = kernel_size[0] * kernel_size[1] * leniency
    cluster_map = conv_result >= threshold
    
    # Label connected components as individual clusters
    labeled_array, num_features = label(cluster_map)
    
    clusters = []
    for i, slice_tuple in enumerate(find_objects(labeled_array)):
        # Find the bounding box of each cluster
        cluster_indices = np.where(labeled_array == i + 1)
        cluster_hits = hit_matrix[cluster_indices].sum()
        midpoint_y = cluster_indices[0].mean()
        midpoint_x = cluster_indices[1].mean()

        clusters.append((midpoint_y, midpoint_x, cluster_hits))
    
    return clusters

kernel_size = (2, 2)
leniency = 0.5

clusters = refine_clusters(hit_matrix[0][0], kernel_size, leniency)
print(clusters)

[(52.5, 6.0, 9)]


Clarification on the functinos
Convolve2d:
2d convolution between 2 2d arrays, it applies a uniform kernel over the hitmatrix
and the kernel sums up the values of hits in each neighborhood definedby its size, 
effectively measuring the local density

Label:
Label identifies and labels connected components within a binary array.
Connected components are set of pixels or array elements that are connected to each other
by pixel adjacency
it analyzes the cluster_map, which are a series True and falses at locations, it labels each contiguous area of activated points by the kernel with unique identifiers

find_objects:
locates and returns a list of slices for each labeled object in an array. Each slice in the list corresponds to a minimal bounding box that contains a labeled object
label was used to connect components, find object then used to find the bounding boxes of these labeled clusters, 
it will be useful in the future, just not now

In [18]:
plot_matrix = np.zeros_like(hit_matrix, dtype=float)

In [19]:
def aggregate_clusters_for_rpc(rpc):
    rpc_plot_matrix = np.zeros_like(rpc[0], dtype=float)

    for event in rpc:
        clusters = refine_clusters(event, kernel_size, leniency)
        
        plot_matrix = np.zeros_like(event, dtype=float)
        
        for cluster in clusters:
            midpoint_y, midpoint_x, cluster_hits = cluster
            y, x = int(midpoint_y), int(midpoint_x)
            plot_matrix[y, x] += cluster_hits 

        rpc_plot_matrix += plot_matrix
    
    return rpc_plot_matrix

def plot_cumulative_heatmap(plot_matrix, rpc_index):
    plt.imshow(plot_matrix, cmap='hot', interpolation='nearest')
    plt.colorbar(label='Cumulative Cluster Hits')
    plt.title(f"RPC {rpc_index} Cumulative Cluster Hits Heatmap")
    plt.xlabel("eta")
    plt.ylabel("phi")
    plt.tight_layout()
    
def plot_cumulative_contour(plot_matrix, rpc_index):
    plt.contourf(plot_matrix, cmap='hot')
    plt.colorbar(label='Cumulative Cluster Hits')
    plt.title(f"RPC {rpc_index} Cumulative Cluster Hits Contour")
    plt.xlabel("eta")
    plt.ylabel("phi")
    plt.tight_layout()
    
def plot_cumulative_3D(plot_matrix, rpc_index):
    x = np.arange(plot_matrix.shape[1])  # Columns
    y = np.arange(plot_matrix.shape[0])  # Rows

    # Create meshgrid for x and y
    x, y = np.meshgrid(x, y)

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot a 3D surface
    surf = ax.plot_surface(x, y, plot_matrix, cmap='viridis')

    # Add color bar which maps values to colors
    fig.colorbar(surf, shrink=0.5, aspect=5)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Height')

    plt.show()
        

In [21]:
print(aggregate_clusters_for_rpc(hit_matrix[0]))

[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  0. ... 16.  0.  0.]
 [ 0.  1.  3. ...  6.  4.  0.]
 [ 0.  0.  0. ...  1.  0.  0.]]


In [35]:
kernel_size = (2, 2)
leniency = 0.8

confirmation = input("This action might cause a large amount of folders to be created at unwanted areas in your computer, please make sure you have changed the pdf directory before proceeding. Are you ready to proceed? (yes/no):")


if confirmation.lower() == 'yes':
    print("Proceeding with the operation...")
    with PdfPages('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\rpc_2D_cumulative_cluster_countour.pdf') as pdf:
        for rpc_index, rpc in enumerate(hit_matrix):
            rpc_plot_matrix = aggregate_clusters_for_rpc(rpc)
            plt.figure(figsize=(10, 8))
            plot_cumulative_contour(rpc_plot_matrix, rpc_index)
            pdf.savefig()
            plt.close()
else:
    print("Operation canceled.")

Proceeding with the operation...
6.0 52.5
16.5 30.5
5.5 36.0
5.5 42.5
20.0 17.5
24.0 61.5
15.0 11.5
15.0 18.5
21.5 18.0
18.0 31.0
12.0 38.0
13.0 22.5
13.0 31.0
13.0 37.5
13.0 53.0
18.5 38.5
23.5 59.0
20.0 55.5
2.0 12.5
8.0 43.0
14.0 45.5
4.0 48.0
17.5 28.5
10.0 58.5
5.0 51.0
5.0 54.0
17.5 57.5
17.5 62.0
23.5 63.0
7.0 9.0
21.0 63.0
1.5 10.0
5.0 10.0
3.0 40.5
3.0 45.0
24.0 62.5
23.5 59.0
1.0 25.0
9.0 14.0
9.0 18.5
8.0 9.5
8.0 18.0
13.5 4.5
13.5 12.0
13.5 39.5
10.0 57.5
24.0 55.0
27.0 37.0
10.0 17.0
25.0 62.0
12.5 28.0
12.5 35.5
12.5 49.0
12.5 59.0
13.0 45.0
27.0 53.0
27.0 61.0
8.0 32.0
21.5 32.0
12.5 8.0
12.5 40.0
20.0 36.0
19.5 53.0
6.5 56.0
6.5 61.0
3.0 57.5
27.0 34.0
16.0 40.0
5.0 62.5
24.0 53.5
16.0 61.5
16.0 35.5
5.5 32.0
18.5 32.0
3.5 56.5
24.0 23.5
10.0 61.0
9.0 18.0
18.0 18.0
23.0 18.0
9.0 25.5
18.0 25.5
23.0 25.5
9.0 30.0
18.0 30.0
23.0 30.0
25.5 20.0
25.5 27.0
25.5 17.0
2.5 58.0
17.0 20.5
21.0 21.0
22.0 28.5
22.0 62.0
3.5 12.0
3.5 17.0
3.5 20.0
4.0 29.5
12.5 29.5
4.0 36.5
12.5 

In [36]:
confirmation = input("This action might cause a large amount of folders to be created at unwanted areas in your computer, please make sure you have changed the pdf directory before proceeding. Are you ready to proceed? (yes/no):")


if confirmation.lower() == 'yes':
    print("Proceeding with the operation...")
    with PdfPages('C:\\Users\\Peter\\OneDrive - University of Cambridge\\Desktop\\Project Excel Work\\PartIIIRPC\\ProAnubis_CERN\\ProAnubis Plots\\rpc_2D_cumulative_cluster_heatmaps.pdf') as pdf:
        for rpc_index, rpc in enumerate(hit_matrix): 
            rpc_plot_matrix = aggregate_clusters_for_rpc(rpc)
            plt.figure(figsize=(10, 8))
            plot_cumulative_heatmap(rpc_plot_matrix, rpc_index)
            pdf.savefig()
            plt.close()

else:
    print("Operation canceled.")

Proceeding with the operation...
6.0 52.5
16.5 30.5
5.5 36.0
5.5 42.5
20.0 17.5
24.0 61.5
15.0 11.5
15.0 18.5
21.5 18.0
18.0 31.0
12.0 38.0
13.0 22.5
13.0 31.0
13.0 37.5
13.0 53.0
18.5 38.5
23.5 59.0
20.0 55.5
2.0 12.5
8.0 43.0
14.0 45.5
4.0 48.0
17.5 28.5
10.0 58.5
5.0 51.0
5.0 54.0
17.5 57.5
17.5 62.0
23.5 63.0
7.0 9.0
21.0 63.0
1.5 10.0
5.0 10.0
3.0 40.5
3.0 45.0
24.0 62.5
23.5 59.0
1.0 25.0
9.0 14.0
9.0 18.5
8.0 9.5
8.0 18.0
13.5 4.5
13.5 12.0
13.5 39.5
10.0 57.5
24.0 55.0
27.0 37.0
10.0 17.0
25.0 62.0
12.5 28.0
12.5 35.5
12.5 49.0
12.5 59.0
13.0 45.0
27.0 53.0
27.0 61.0
8.0 32.0
21.5 32.0
12.5 8.0
12.5 40.0
20.0 36.0
19.5 53.0
6.5 56.0
6.5 61.0
3.0 57.5
27.0 34.0
16.0 40.0
5.0 62.5
24.0 53.5
16.0 61.5
16.0 35.5
5.5 32.0
18.5 32.0
3.5 56.5
24.0 23.5
10.0 61.0
9.0 18.0
18.0 18.0
23.0 18.0
9.0 25.5
18.0 25.5
23.0 25.5
9.0 30.0
18.0 30.0
23.0 30.0
25.5 20.0
25.5 27.0
25.5 17.0
2.5 58.0
17.0 20.5
21.0 21.0
22.0 28.5
22.0 62.0
3.5 12.0
3.5 17.0
3.5 20.0
4.0 29.5
12.5 29.5
4.0 36.5
12.5 

This heat map is not very useful since the clarity of the map is not easy to see. There could be a much easier way to display this graph. So just trying out if plotting in 3D will actually help